In [ ]:
import pandas as pd
import os
import tensorflow as tf


dff= pd.read_csv("MSFT.csv")

print("Starting file:")
print(dff[0:10])

print("Ending file:")
print(dff[-10:])

Starting file:
         Date      Open      High       Low     Close  Adj Close      Volume
0  1986-03-13  0.088542  0.101563  0.088542  0.097222   0.060163  1031788800
1  1986-03-14  0.097222  0.102431  0.097222  0.100694   0.062311   308160000
2  1986-03-17  0.100694  0.103299  0.100694  0.102431   0.063386   133171200
3  1986-03-18  0.102431  0.103299  0.098958  0.099826   0.061774    67766400
4  1986-03-19  0.099826  0.100694  0.097222  0.098090   0.060700    47894400
5  1986-03-20  0.098090  0.098090  0.094618  0.095486   0.059089    58435200
6  1986-03-21  0.095486  0.097222  0.091146  0.092882   0.057477    59990400
7  1986-03-24  0.092882  0.092882  0.089410  0.090278   0.055866    65289600
8  1986-03-25  0.090278  0.092014  0.089410  0.092014   0.056940    32083200
9  1986-03-26  0.092014  0.095486  0.091146  0.094618   0.058551    22752000
Ending file:
            Date        Open        High         Low       Close   Adj Close  \
9634  2024-06-05  417.809998  424.079987  416

In [ ]:
df= dff[['Close']]  # Select  'Close' columns
df.head()

,Close
0,0.097222
1,0.100694
2,0.102431
3,0.099826
4,0.098090


In [ ]:
# Define the split time
split_time =-707
df_train = dff['Close'][:split_time]
df_valid = dff['Close'][split_time:-31].reset_index(drop=True)

df_valid

0      299.089996
1      299.720001
2      303.589996
3      301.880005
4      301.829987
          ...    
671    402.250000
672    389.329987
673    394.940002
674    397.839996
675    406.660004
Name: Close, Length: 676, dtype: float64

In [ ]:
import numpy as np

def to_sequences(seq_size, obs):
    x = []
    y = []

    for i in range(len(obs)-SEQUENCE_SIZE):
        #print(i)
        window = obs[i:(i+SEQUENCE_SIZE)]
        after_window = obs[i+SEQUENCE_SIZE]
        window = [[x] for x in window]
        #print("{} - {}".format(window,after_window))
        x.append(window)
        y.append(after_window)

    return np.array(x),np.array(y)


SEQUENCE_SIZE = 30
x_train,y_train = to_sequences(SEQUENCE_SIZE,df_train)
x_test,y_test = to_sequences(SEQUENCE_SIZE,df_valid)

print("Shape of training set: {}".format(x_train.shape))
print("Shape of test set: {}".format(x_test.shape))

Shape of training set: (8907, 30, 1)
Shape of test set: (646, 30, 1)


In [ ]:
print(x_train.shape)


(8907, 30, 1)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

In [ ]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.2,
    dropout=0.2,
)



model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.Adam(learning_rate=2e-5),
)



In [ ]:
callbacks = [keras.callbacks.EarlyStopping(patience=10, \
    restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

Epoch 1/200
112/112 [==============================] - 77s 600ms/step - loss: 459.4659 - val_loss: 6775.3130
Epoch 2/200
112/112 [==============================] - 64s 574ms/step - loss: 150.4603 - val_loss: 1449.6218
Epoch 3/200
112/112 [==============================] - 63s 561ms/step - loss: 66.5775 - val_loss: 249.0223
Epoch 4/200
112/112 [==============================] - 63s 557ms/step - loss: 52.6093 - val_loss: 122.6515
Epoch 5/200
112/112 [==============================] - 62s 554ms/step - loss: 51.3907 - val_loss: 103.2530
Epoch 6/200
112/112 [==============================] - 63s 559ms/step - loss: 49.8273 - val_loss: 89.0434
Epoch 7/200
112/112 [==============================] - 61s 547ms/step - loss: 51.0723 - val_loss: 69.5039
Epoch 8/200
112/112 [==============================] - 67s 597ms/step - loss: 47.2247 - val_loss: 61.7298
Epoch 9/200
112/112 [==============================] - 63s 560ms/step - loss: 44.9612 - val_loss: 50.9400
Epoch 10/200
112/112 [===============

392.8635559082031

In [ ]:
from sklearn import metrics

pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))

21/21 [==============================] - 2s 90ms/step
Score (RMSE): 19.820786131253943


In [ ]:
#FORECASTING

In [ ]:
# Define the split time
split_time =-32
df_train = dff['Close'][:split_time]
df_valid = dff['Close'][split_time:-1].reset_index(drop=True)

df_valid

0     406.660004
1     413.540009
2     409.339996
3     410.540009
4     412.320007
5     414.739990
6     413.720001
7     416.559998
8     423.079987
9     420.989990
10    420.209991
11    425.339996
12    429.040009
13    430.519989
14    427.000000
15    430.160004
16    430.320007
17    429.170013
18    414.670013
19    415.130005
20    413.519989
21    416.070007
22    424.010010
23    424.519989
24    423.850006
25    427.869995
26    432.679993
27    441.059998
28    441.579987
29    442.570007
30    448.369995
Name: Close, dtype: float64

In [ ]:
import numpy as np

def to_sequences(seq_size, obs):
    x = []
    y = []

    for i in range(len(obs)-SEQUENCE_SIZE):
        #print(i)
        window = obs[i:(i+SEQUENCE_SIZE)]
        after_window = obs[i+SEQUENCE_SIZE]
        window = [[x] for x in window]
        #print("{} - {}".format(window,after_window))
        x.append(window)
        y.append(after_window)

    return np.array(x),np.array(y)


SEQUENCE_SIZE = 30
x_train,y_train = to_sequences(SEQUENCE_SIZE,df_train)
x_test,y_test = to_sequences(SEQUENCE_SIZE,df_valid)

print("Shape of training set: {}".format(x_train.shape))
print("Shape of test set: {}".format(x_test.shape))

Shape of training set: (9582, 30, 1)
Shape of test set: (1, 30, 1)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

In [ ]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.2,
    dropout=0.2,
)



model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.Adam(learning_rate=2e-5),
)



In [ ]:
callbacks = [keras.callbacks.EarlyStopping(patience=10, \
    restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)


Epoch 1/200
120/120 [==============================] - 80s 592ms/step - loss: 473.9762 - val_loss: 13441.9346
Epoch 2/200
120/120 [==============================] - 71s 593ms/step - loss: 149.0647 - val_loss: 2021.0980
Epoch 3/200
120/120 [==============================] - 70s 588ms/step - loss: 77.3945 - val_loss: 354.1902
Epoch 4/200
120/120 [==============================] - 70s 581ms/step - loss: 75.1361 - val_loss: 182.6381
Epoch 5/200
120/120 [==============================] - 71s 589ms/step - loss: 66.9288 - val_loss: 140.2717
Epoch 6/200
120/120 [==============================] - 70s 585ms/step - loss: 68.4381 - val_loss: 120.8968
Epoch 7/200
120/120 [==============================] - 73s 608ms/step - loss: 65.6559 - val_loss: 100.4804
Epoch 8/200
120/120 [==============================] - 72s 599ms/step - loss: 63.2184 - val_loss: 119.7105
Epoch 9/200
120/120 [==============================] - 67s 553ms/step - loss: 61.1876 - val_loss: 114.6646
Epoch 10/200
120/120 [==========

In [ ]:
def iterative_forecast(model, initial_input, steps=30):
    forecast = []
    current_input = initial_input.reshape(1, -1, 1)  # Reshape to (1, time steps, features)

    for _ in range(steps):
        prediction = model.predict(current_input)[0, 0]  # Remove extra dimension after prediction
        forecast.append(prediction)

        # Update the input for the next prediction
        current_input = np.roll(current_input, -1, axis=1) # Shift the input one step back along time axis
        current_input[0, -1, 0] = prediction # Replace the last value with the prediction


    return np.array(forecast)

# Get the most recent data to start the prediction
from sklearn.preprocessing import MinMaxScaler

# Scale data
#scaler = MinMaxScaler()
#x_train= scaler.fit_transform(x_train.reshape(-1, 1)).reshape(x_train.shape)
#y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(y_train.shape)

initial_input = x_train[-1, -input_shape[0]:]

#Perform the iterative forecast
forecast = iterative_forecast(model, initial_input, steps=30)
print(forecast)
# Denormalize the predictions
#forecast_1 = forecast.reshape(-1, 1)
#denormalized_predictions = scaler.inverse_transform(forecast_1)
#print(denormalized_predictions)

1/1 [==============================] - 0s 28ms/step
[425.62592 428.92725 426.3989  427.5403  429.56458 432.30005 428.92633
 428.7851  431.60538 431.7228  420.03497 413.74606 419.22107 420.25577
 417.9411  420.878   426.91214 431.80255 427.2162  420.88864 426.94772
 432.38016 425.89236 423.25394 433.454   434.1812  425.08005 419.2269
 427.17767 436.62628]


In [ ]:
forecast = forecast.squeeze()  # Removes the single-dimensional entries from the shape of an array.
x_test = x_test.squeeze()


In [ ]:
|print(tf.keras.metrics.mean_squared_error(x_test, forecast).numpy())
print(tf.keras.metrics.mean_absolute_error(x_test, forecast).numpy())

147.5022
10.767515
